In [ ]:
import datetime as dt
import csv
import pandas as pd
import networkx as nx

In [ ]:
vaccine_terms = 'shot|vaccine|vacine|vacines|vaccines|vaccinate|vaccination|vaccinations|vaccinated|vaccinating|vaxxed|vaxx|vax|\
unvaccinated|unvaxxed|antivaxx|antivaccination|anti|\
moderna|pfizer|J&J|immune|immunize|immunizes|immunized|immunization|immunizations'
vaccine_terms_list= vaccine_terms.split('|')

In [55]:
data_path = '/content/drive/MyDrive/math168_project/data/08_31.csv'

In [56]:
import os
import glob
  
# csv_files = glob.glob(os.path.join(data_path, "*.csv"))

# read the csv file individually
df = pd.read_csv(data_path, dtype="str")
      
# print the location and filename
df = df[['user_id', 'text', 'created_at', 'id', 'in_reply_to_screen_name', 'retweet_screen_name', 'user_screen_name']]
    
# either a reply or a retweet
df = df.dropna(subset=['in_reply_to_screen_name', 'retweet_screen_name'], how='all')
df = df[df.text.str.contains(vaccine_terms)]
    
df['target_user'] = df['in_reply_to_screen_name'].fillna('') + df['retweet_screen_name'].fillna('')
    
# make graph
# G = nx.from_pandas_edgelist(df, 'user_screen_name', 'target_user', create_using=nx.MultiGraph())
    
# save the updated csv to the same directory - decide on a naming convention for these files
# currently it is {original_file_name}_twitter.csv
f_name, _ = os.path.splitext(data_path)
df.to_csv(f_name+'_twitter.csv', index=False, encoding='utf-8-sig')



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [61]:
# merge all updated data together
# get all csv_files in the directory
data_path2 = '/content/drive/MyDrive/math168_project/data/'
csv_files = glob.glob(os.path.join(data_path2, "*.csv"))

# get all updated dataframes
files = []
for file in csv_files:
  if 'twitter' in file: 
    files.append(file)

# merge
completed_df = pd.concat(map(pd.read_csv, files), ignore_index=True)

# save the completed csv file to the same directory
completed_df.to_csv(data_path2+'completed_twitter_data.csv', index=False, encoding='utf-8-sig')

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py:304: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  sort=sort,
/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py:304: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  sort=sort,


In [62]:
len(completed_df)

2967844